In [30]:
import pandas as pd #для анализа и предобработки данных
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing  import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder #для кодирования признаков
from sklearn.linear_model import LinearRegression #линейные модели
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

from geopy.geocoders import Nominatim
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [31]:
data = pd.read_excel('data/data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [32]:
#производим предобработку данных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

In [33]:
#выделяем целевую переменную
y = data['price']
x = data.drop(columns='price')

In [34]:
#разделяем наши данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [35]:
#обучаем модель на всех признаках
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


In [36]:
#производим отбор лучших признаков при помощи метода рекурсивного исключения признаков
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=3, step=1)
selector_rfe = selector.fit(X_train, y_train)
 
selector_rfe.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [37]:
#производим отбор лучших признаков при помощи метода выбора k лучших переменных
selector_skb = SelectKBest(f_regression, k=3)
selector_skb.fit(X_train, y_train)
 
selector_skb.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [40]:
#оставляем в данных признаки, полученные при помощи метода рекурсивного исключения признаков
X_train_rfe = X_train[['year', 'cylinders', 'lat']]
X_test_rfe = X_test[['year', 'cylinders', 'lat']]

In [41]:
#обучаем модель на признаках, полученных при помощи метода рекурсивного исключения признаков
model_rfe = LinearRegression()
model_rfe.fit(X_train_rfe, y_train)
y_predicted_rfe = model_rfe.predict(X_test_rfe)
 
mae = mean_absolute_error(y_test, y_predicted_rfe)
print('MAE RFE model: %.3f' % mae)

MAE RFE model: 5096.570


In [42]:
#оставляем в данных признаки, полученные при помощи метода выбора k лучших переменных
X_train_skb = X_train[['year', 'cylinders', 'odometer']]
X_test_skb = X_test[['year', 'cylinders', 'odometer']]

In [44]:
#обучаем модель на признаках, полученных при помощи метода выбора k лучших переменных
model_skb = LinearRegression()
model_skb.fit(X_train_skb, y_train)
y_predicted_skb = model_skb.predict(X_test_skb)
 
mae = mean_absolute_error(y_test, y_predicted_skb)
print('MAE RFE model: %.3f' % mae)

MAE RFE model: 4708.946


По результатам сравнения метрик, полученных в ходе обучения моделей на признаках отобранных при помощи 
методов рекурсивного исключения признаков и выбора k лучших переменных, можно сделать вывод, что 
модель, построенная при использовании метода выбора k лучших переменных, показала себя лучше. 
Метрика MAE у данной модели составила 4708.946 против MAE 5096.570 у другой модели.
Однако также стоит отметить, что обе модели справились с предсказанием хуже, 
чем модель, простроенная на всех признаках, которая показала результат MAE 4682.957.